In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
# import openpyxl
from openpyxl import load_workbook

In [2]:

wb = load_workbook("datasets/hindi.xlsx")
sheet = wb.active
col = csv.writer(open("tt.csv", 'w', newline=""))
for r in sheet.rows:
    col.writerow([cell.value for cell in r])
  
df = pd.DataFrame(pd.read_csv("tt.csv"))
df.to_csv("datasets/hindi.csv", sep=",")
df.head(5)

,text,task1
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HATE
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NOT_HATE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HATE
3,Yey nina khothani labafazi benu phambili Finis...,HATE
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HATE


In [27]:
df['task1'] = df['task1'].map({'HATE':1, 'NOT_HATE':0})
df.head()
print(df["text"][2])

RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ?

ये आज़मगढ़ है और यहाँ से "टोटी चोर"…


In [4]:
import collections

with open("datasets/Hindi_StopWords.txt",encoding='utf-8') as f:
    stopword= f.read().strip('\ufeff')
stopword = stopword.split(", ")
stopword = [i.strip("'") for i in stopword]

stopwords = set(stopword)
print(stopwords)

{'तेरा', 'बहि', 'अदि', 'उन', 'अपनी', 'द्वारा', 'निहायत', 'यहां', 'तुझे', 'इसी', 'साभ', 'ने', 'हें', 'करना', 'इसके', 'कोइ', 'उन्ह', 'मेरी', 'तिंहों', 'आदि', 'वहाँ', 'कोनसा', 'तिन्हों', 'था', 'जिन्हें', 'लिए', 'वे', 'ओर', 'सो', 'किन्हें', 'मगर', 'कोन', 'किंहों', 'तू', 'वर्ग', 'अभी', 'काफि', 'सब', 'वुह', 'पास', 'एसे', 'उसे', 'मैं', 'उसका', 'हूं', 'कहते', 'किन्हों', 'ये', 'जब', 'जिंहों', 'लिये', 'होती', 'तेरी', 'इन', 'इसका', 'इस', 'जिस', 'अपने', 'मानो', 'एवं', 'रवासा', 'व', 'उसने', 'जा', 'अभि', 'जिंहें', 'का', 'काफ़ी', 'आप', 'जहां', 'तक', 'वहीं', 'यदि', 'जेसा', 'वह ', 'ले', 'निचे', 'इंहिं', 'एस', 'बिलकुल', 'संग', 'उसकी', 'भितर', 'अंदर', 'रखें', 'में', 'कहता', 'वाले', 'इंहें', 'कुल', 'कुछ', 'इसे', 'दुसरा', 'लेकिन', 'रह', 'हुआ', 'बही', 'उन्हीं', 'दिया', 'उनको', 'इसलिये', 'हम', 'वह', 'कौन', 'जो', 'तब', 'हैं', 'कइ', 'कि', 'उसके', 'वहां', 'करते', 'उनका', 'कर', 'साथ', 'यिह', 'या', "इतयादि' ,'यहाँ", 'है', 'वहिं', 'गए', 'नहीं', 'दो', 'कई', 'अपना', 'जिसे', 'कह', 'उंहिं', 'जैसे', 'करें', 'मुझे', 'बाल

In [5]:
import itertools

hate_det = df[df.task1==1]
hate_det.reset_index(drop=True, inplace=True)
acc_det = df[df.task1==0]
acc_det.reset_index(drop=True, inplace=True)

# Tokenization
hate_news = []
for rows in range(0, hate_det.shape[0]):
    head_txt = hate_det.text[rows]
    head_txt = head_txt.split(" ")
    hate_news.append(head_txt)

hate_list = list(itertools.chain(*hate_news))
print(hate_list[1:20])
print("\n" + "Number of words: " + str(len(hate_list)))
acc_news = []
for rows in range(0, acc_det.shape[0]):
    head_txt = acc_det.text[rows]
    head_txt = head_txt.split(" ")
    acc_news.append(head_txt)

acc_list = list(itertools.chain(*acc_news))
print("\n" + str(acc_list[1:20]))
print("\n" + "Number of words: " + str(len(acc_list)))

['आदमीं', 'को', 'मारने', 'पर', 'गोडसे', 'आतंकी', 'हो', 'सके', 'है', 'तो\n17000', 'सिखो,', '5000', 'भोपाली,', '3000', 'तमिलों', 'का', 'कत्लेआम', 'करवाने', 'वाला']

Number of words: 20755

['@Vishesh4:', '@jawaharyadavbjp', 'जवाहर', 'यादव,', 'अगर', 'हिम्मत', 'है', 'तो', 'पूरा', 'वीडियो', 'डालो', 'फिर', 'तुम्हारे', 'झूठ', 'सामने', 'आएंगे।\nदीपेंदर', 'ने', 'साफ', 'कहा,']

Number of words: 46752


In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [8]:
X = df.text
Y = df.task1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [11]:
tk = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
tk.fit_on_texts(X_train)
seqs = tk.texts_to_sequences(X_train)
max_len = 100
seqs_mat = tf.keras.preprocessing.sequence.pad_sequences(seqs,maxlen=max_len)
X_train[1:5]

2537    नहीं कोई बात होती फिर भी बात होती है,\nकुछ रिश...
1384    RT @Aastha9305: बनारस से भी ज्यादा दिलचस्प सीट...
1301    RT @RoflNana_: डाक्टर आप को डॉक्टर बन्ने के लि...
372     जो नाले से गैस बत्तख से ऑक्सीजन और गोबर से कोह...
Name: text, dtype: object

In [12]:
def model_def():
    inputs = tf.keras.layers.Input(name='inputs', shape=[max_len])
    layer = tf.keras.layers.Embedding(1000,50,input_length=max_len)(inputs)
    layer = tf.keras.layers.LSTM(64)(layer)
    layer = tf.keras.layers.Dense(256,name='FC1')(layer)
    layer = tf.keras.layers.Activation('relu')(layer)
    layer = tf.keras.layers.Dropout(0.2)(layer)
    layer = tf.keras.layers.Dense(1,name='out_layer')(layer)
    
    model = tf.keras.models.Model(inputs=inputs,outputs=layer)
    return model

In [13]:
model = model_def()
model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_______________________

In [14]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_acc', 
    mode='max',
    patience=10
)
# increase epochs and other steps for better training
model.fit(seqs_mat,Y_train,batch_size=100,epochs=10,
          validation_split=0.1,callbacks=[es])

Train on 2124 samples, validate on 236 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
2124/2124 [==============================] - 12s 6ms/sample - loss: 0.6867 - acc: 0.7024 - val_loss: 0.5965 - val_acc: 0.7076
Epoch 2/10
2124/2124 [==============================] - 9s 4ms/sample - loss: 0.5931 - acc: 0.7331 - val_loss: 0.5948 - val_acc: 0.6907
Epoch 3/10
2124/2124 [==============================] - 9s 4ms/sample - loss: 0.4769 - acc: 0.7830 - val_loss: 0.6899 - val_acc: 0.6780
Epoch 4/10
2124/2124 [==============================] - 9s 4ms/sample - loss: 0.3862 - acc: 0.8475 - val_loss: 1.0268 - val_acc: 0.6695
Epoch 5/10
2124/2124 [==============================] - 9s 4ms/sample - loss: 0.4306 - acc: 0.8413 - val_loss: 2.9805 - val_acc: 0.5297
Epoch 6/10
2124/2124 [==============================] - 9s 4ms/sample - loss: 0.5291 - acc: 0.7830 - val_loss: 0.7894 - val_acc: 0.6186
Epoch 7/10
2124/2124 [====================

In [15]:
test_sequences = tk.texts_to_sequences(X_test)
test_sequences_matrix = tf.keras.preprocessing.sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

591/591 [==============================] - 2s 3ms/sample - loss: 2.0797 - acc: 0.6734
Test set
  Loss: 2.080
  Accuracy: 0.673


In [16]:
def predict_hate(user_seq):
#     prediction
    prob = model.predict(user_seq)
    probability = np.mean(prob, axis=0)

    if probability > 0.5:
        return("Hate")
    elif probability < 0.5:
        return("Not Hate")
    elif probability == 0.5:
        return("Neutral")

def user_text_processing(user_text):
    user_text = user_text.split()
    user_text = [word.lower() for word in user_text if word not in stopwords]
#     user_text = [lemm.lemmatize(word) for word in user_text]
    user_text
    user_seq = np.array(user_text)
    user_seq = tk.texts_to_sequences(user_seq)
    user_seq = tf.keras.preprocessing.sequence.pad_sequences(user_seq,maxlen=max_len)

    return user_seq

In [29]:
user_text = 'RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ? ये आज़मगढ़ है और यहाँ से "टोटी चोर"…'
user_seq = user_text_processing(user_text)
user_seq
prediction = predict_hate(user_seq)
print(f"Sentence '{user_text}' is of '{prediction}' nature")


Sentence 'RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ? ये आज़मगढ़ है और यहाँ से "टोटी चोर"…' is of 'Not Hate' nature
